In [1]:
import pandas as pd
import datetime
import sys
import matplotlib.pyplot as plt
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import seaborn as sns
import numpy as np
sys.path.append('..')

# Read in ChargePoint/ViriCiti data

In [2]:
vc_file = '../beb_chargers/data/viriciti/mar24_energy_data.csv'
vc_df = pd.read_csv(vc_file)
vc_df = vc_df.astype(
    dtype={
        'Name': str,
        'ISO Time': str
    }
)
# Convert time column to datetime and make sure it's the right time zone
vc_df['ISO Time'] = pd.to_datetime(vc_df['ISO Time'], utc=True).dt.tz_convert('US/Pacific')
# Identify all vehicle IDs observed
vids = vc_df['Name'].unique()

# Read in GTFS-realtime data

In [5]:
rt_prefix = '../beb_chargers/data/realtime/metro'
rt_files = sorted([f for f in os.listdir(rt_prefix) if f[-4:]=='.pkl'])
df_list = list()
for f in rt_files:
    fname = rt_prefix + '/' + f
    # Read in realtime data provided by Zack
    df = pd.read_pickle(fname).reset_index(drop=True)
    df = df.astype(
        dtype={
            'vehicle_id': str,
            'scheduleDeviation': int,
            'trip_id': str
        }
    )
    # Convert time column to datetime and change time zone
    df['locationtime'] = pd.to_datetime(
        df['locationtime'].astype(int), unit='s', utc=True).dt.tz_convert('US/Pacific')
    
    # Filter down realtime data to only include buses in ChargePoint data
    df = df[df['vehicle_id'].isin(vids)]
    df = df.drop(columns=['orientation'])
    df_list.append(df)
rt_df = pd.concat(df_list).reset_index()
# Add a date column for grouping data by day
rt_df['date'] = pd.to_datetime(rt_df['locationtime'].dt.date)

# Read in static GTFS data

In [7]:
# Read in GTFS stop times
stop_times_df = pd.read_csv('../beb_chargers/data/gtfs/metro_mar24/stop_times.txt')
stop_times_df = stop_times_df.astype(
    {'trip_id': str, 'stop_id': str}
)
# Filter down to included trip IDs
stop_times_df = stop_times_df[stop_times_df['trip_id'].isin(rt_df['trip_id'].unique())]
# Convert to timedeltas
stop_times_df['departure_timedelta'] = pd.to_timedelta(stop_times_df['departure_time'])

# Trips file (gives us shape and route IDs)
trips_df = pd.read_csv('../beb_chargers/data/gtfs/metro_mar24/trips.txt')
trips_df = trips_df.astype(
    {'trip_id': str, 'shape_id': str, 'route_id': str}
)
trips_df = trips_df[trips_df['trip_id'].isin(rt_df['trip_id'].unique())]

# Routes file (gives us route names)
routes_df = pd.read_csv('../beb_chargers/data/gtfs/metro_mar24/routes.txt')
routes_df = routes_df.astype(
    {'route_id': str}
)

In [8]:
# Add stop info to realtime
rt_df = rt_df.merge(
    stop_times_df.drop(
        columns=[
            'arrival_time', 'departure_time', 'stop_headsign', 'pickup_type', 'drop_off_type', 'shape_dist_traveled', 'timepoint']
    ),
    left_on=['trip_id', 'nextStop'], right_on=['trip_id', 'stop_id']
)
rt_df['departure_time'] = (rt_df['date'] + rt_df['departure_timedelta']).dt.tz_localize('US/Pacific')

# Clean realtime data
Remove outliers and identify start/end times of all trips

In [10]:
clean_count = 0
error_count = 0
plot_ix = 0
dates = list()
tids = list()
start_times = list()
end_times = list()
start_delays = list()
end_delays = list()
veh_ids = list()
sort_by = 'locationtime'  # ['stop_sequence', 'locationtime']
all_dates = rt_df['date'].unique()
error_dfs = list()
for d in all_dates:
    d_df = rt_df[rt_df['date'] == d]
    all_trips = d_df['trip_id'].unique()
    for t in all_trips:
        t_df = d_df[d_df['trip_id'] == t].sort_values(by=sort_by)

        # Exclude temporal outliers with interquantile filter
        q1 = t_df['locationtime'].quantile(.25)
        q3 = t_df['locationtime'].quantile(.75)
        iqr = q3 - q1
        t_df = t_df[
            t_df['locationtime'].between(
                q1-0.75*iqr,
                q3+0.75*iqr
            )
        ]

        # Another IQR filter for schedule deviation
        q1 = t_df['scheduleDeviation'].quantile(.25)
        q3 = t_df['scheduleDeviation'].quantile(.75)
        iqr = q3 - q1
        t_df = t_df[
            t_df['scheduleDeviation'].between(
                q1 - 1.5*iqr,
                q3 + 1.5*iqr
            )
        ]
        
        # Exclude all data that includes big negative schedule deviations
        t_df = t_df[t_df['scheduleDeviation'] > -900]
        
        
        try:
            t_df = t_df.sort_values(by=['stop_sequence', 'locationtime'])
            first_stop_df = t_df.groupby('nextStop').tail(1).iloc[0]
            first_stop_delay = first_stop_df['scheduleDeviation']
            first_recorded_time = first_stop_df['locationtime']
            veh_id = first_stop_df['vehicle_id']
            last_recorded_time = t_df.groupby('nextStop').head(1).iloc[-1]['locationtime']
            last_stop_delay = t_df.groupby('nextStop').head(1).iloc[-1]['scheduleDeviation']
            
            start_times.append(first_recorded_time)
            end_times.append(last_recorded_time)
            start_delays.append(first_stop_delay)
            end_delays.append(last_stop_delay)
            veh_ids.append(veh_id)
            tids.append(t)
            dates.append(d)
            clean_count += 1
            
        except IndexError as e:
            error_count += 1
            # We end up here if the DF is empty after filtering
            t_df = d_df[d_df['trip_id'] == t].sort_values(by='locationtime')
            error_dfs.append(t_df)
            fig = px.line(t_df, x='locationtime', y='scheduleDeviation', title='{}, Trip {}'.format(
                datetime.datetime.strftime(d, '%m/%d/%Y'), t))
            fig.write_image('images/error_delay_by_trip_{}.pdf'.format(plot_ix))
            plot_ix += 1

rt_trip_summary = pd.DataFrame(
    {
        'date': dates,
        'vehicle_id': veh_ids,
        'trip_id': tids,
        'first_recorded_time': start_times,
        'last_recorded_time': end_times,
        'delay_at_start': start_delays,
        'delay_at_end': end_delays
    }
).sort_values(by=['vehicle_id', 'first_recorded_time'])
print(error_count, clean_count)

# Only include trips at least 10 minutes long
rt_trip_summary['recorded_duration'] = rt_trip_summary['last_recorded_time'] - rt_trip_summary['first_recorded_time']
rt_trip_summary = rt_trip_summary[rt_trip_summary['recorded_duration'].dt.total_seconds() > 600]

In [14]:
rt_trip_summary['first_recorded_time'].dt.hour.value_counts().sort_index()

## Filter down the data to exclude any points outside the recorded bounds
We don't really use the raw data any further, so this is a bit unnecessary, but validates our approach above.

In [17]:
def get_trip_realtime(realtime_df, tid, min_time=None, max_time=None):
    """
    Get the realtime data associated with the given vehicle ID
    within the time range [min_time, max_time].
    """
    t_df = realtime_df[realtime_df['trip_id'] == tid]
    if min_time is not None:
        t_df = t_df[t_df['locationtime'] >= min_time]
    if max_time is not None:
        t_df = t_df[t_df['locationtime'] <= max_time]

    return t_df.sort_values(by='locationtime')

In [18]:
# This code is clunky and slow, but gets the job done
trip_dfs = list()
trip_summary = rt_trip_summary.set_index(['date', 'trip_id'])
for ix in trip_summary.index:
    t_df = get_trip_realtime(rt_df, ix[1], trip_summary.loc[ix, 'first_recorded_time'], trip_summary.loc[ix, 'last_recorded_time'])
    # Run IQR filter again on cleaned data
    q1 = t_df['locationtime'].quantile(.25)
    q3 = t_df['locationtime'].quantile(.75)
    iqr = q3 - q1
    t_df = t_df[
        t_df['locationtime'].between(
            q1-0.75*iqr,
            q3+0.75*iqr
        )
    ]
    
    trip_dfs.append(t_df)
rt_cleaned = pd.concat(trip_dfs).sort_values(by='locationtime')

In [19]:
len(rt_df), len(rt_cleaned)

# Visualize the results of data cleaning

## Show some problematic data prior to cleaning

In [10]:
n = 0
bad_vehs = rt_df.set_index(['date', 'trip_id']).reset_index()[['date', 'vehicle_id']].drop_duplicates()
for ix, rw in bad_vehs.iterrows():
    n += 1
    # n > 20
    if n > 5:
        break
    veh_df = rt_df[
        (rt_df['date'] == rw['date']) & (rt_df['vehicle_id'] == rw['vehicle_id'])
    ]
    fig = px.scatter(
        veh_df, x='locationtime', y='scheduleDeviation', color='trip_id'
    )
    date_str = rw.date.strftime('%-m/%-d/%Y')
    fig.update_layout(
        title_text=f'Bus {rw.vehicle_id} on {date_str}'
    )
    if rw['vehicle_id'] == '4712' and rw['date'] == datetime.datetime(2024, 3, 3):
        fig.write_image('raw_4712.pdf')

    fig.show()

In [11]:
veh_df = rt_df[
    (rt_df['date'] == '03-03-2024') & (rt_df['vehicle_id'] == '4712')
]
# veh_df = veh_df[veh_df['locationtime'] <= '2024-03-01 08:18:43-08:00']
fig = px.scatter(
        veh_df, x='locationtime', y='scheduleDeviation', color='trip_id'
    )
date_str = '3/3/2024'
# date_str = rw.date.strftime('%-m/%-d/%Y')
fig.update_layout(
    title_text=f'Bus 4712 on {date_str}',
    legend_title_text='Trip ID'
)
fig.show()
fig.write_image('rt_trip_id_problem.pdf')


## Show the different types of outliers

In [12]:
veh_df = rt_df[
    (rt_df['date'] == '03-02-2024') & (rt_df['trip_id'] == '635411515')
]
veh_df = veh_df.copy()
veh_df['Outlier Type'] = 'Not outlier'

q1 = veh_df['scheduleDeviation'].quantile(0.25)
q3 = veh_df['scheduleDeviation'].quantile(0.75)
iqr = q3 - q1
veh_df.loc[~veh_df['scheduleDeviation'].between(q1 - 1.5*iqr, q3 + 1.5*iqr), 'Outlier Type'] = 'Schedule deviation'

q1 = veh_df['locationtime'].quantile(0.25)
q3 = veh_df['locationtime'].quantile(0.75)
iqr = q3 - q1
veh_df.loc[~veh_df['locationtime'].between(q1 - 0.75*iqr, q3 + 0.75*iqr), 'Outlier Type'] = 'Sample time'

fig = px.scatter(
    veh_df, x='locationtime', y='scheduleDeviation', color='Outlier Type'
)
date_str = '3/2/2024'
# date_str = rw.date.strftime('%-m/%-d/%Y')
fig.update_layout(
    title_text=f'Bus 4800, Trip 635411515 on {date_str}'
)
fig.show()
fig.write_image('rt_outliers.pdf')

## Show the data after cleaning

In [13]:
veh_df = rt_cleaned[
    (rt_cleaned['date'] == '03-03-2024') & (rt_cleaned['vehicle_id'] == '4712')
]
# veh_df = veh_df[veh_df['locationtime'] <= '2024-03-01 08:18:43-08:00']
fig = px.scatter(
        veh_df, x='locationtime', y='scheduleDeviation', color='trip_id'
    )
date_str = '3/3/2024'
fig.update_layout(
    title_text=f'Bus 4712 on {date_str}',
    legend_title_text='Trip ID'
)
fig.show()
fig.write_image('rt_trip_id_after_clean.pdf')

In [14]:
n = 0
# bad_vehs = rt_cleaned.set_index(['date', 'trip_id']).loc[bad_ixs, :].reset_index()[['date', 'vehicle_id']].drop_duplicates()
for ix, rw in bad_vehs.iterrows():
    n += 1
    if n > 5:
        break
    veh_df = rt_cleaned[
        (rt_cleaned['date'] == rw['date']) & (rt_cleaned['vehicle_id'] == rw['vehicle_id'])
    ]
    fig = px.scatter(
        veh_df, x='locationtime', y='scheduleDeviation', color='trip_id'
    )
    date_str = rw.date.strftime('%-m/%-d/%Y')
    fig.update_layout(
        title_text=f'Bus {rw.vehicle_id} on {date_str}'
    )
    if rw['vehicle_id'] == '4712' and rw['date'] == datetime.datetime(2024, 3, 3):
        fig.write_image('cleaned_4712.pdf')
    fig.show()

# Process realtime data to estimate trip durations

## Summarize scheduled times from static GTFS

In [20]:
scheduled_times = pd.DataFrame()
scheduled_times['start_time_sched'] = stop_times_df.sort_values(
    by=['trip_id', 'departure_time']).groupby('trip_id')[
['trip_id', 'departure_time']].head(1).set_index('trip_id')
scheduled_times['end_time_sched'] = stop_times_df.sort_values(
    by=['trip_id', 'departure_time']).groupby('trip_id')[
['trip_id', 'departure_time']].tail(1).set_index('trip_id')
scheduled_times['duration_sched'] = (
    pd.to_timedelta(scheduled_times['end_time_sched']) - pd.to_timedelta(scheduled_times['start_time_sched'])
).dt.total_seconds()

In [22]:
# Add scheduled duration column
rt_trip_summary = rt_trip_summary.merge(
    scheduled_times[['start_time_sched', 'end_time_sched', 'duration_sched']], 
    left_on='trip_id', right_index=True
)

In [23]:
rt_trip_summary['start_time_sched'] = rt_trip_summary['date'] + pd.to_timedelta(rt_trip_summary['start_time_sched'])
rt_trip_summary['end_time_sched'] = rt_trip_summary['date'] + pd.to_timedelta(rt_trip_summary['end_time_sched'])
rt_trip_summary['start_time_actual'] = rt_trip_summary['start_time_sched'] + pd.to_timedelta(
    rt_trip_summary['delay_at_start'], unit='s'
)
rt_trip_summary['end_time_actual'] = rt_trip_summary['end_time_sched'] + pd.to_timedelta(
    rt_trip_summary['delay_at_end'], unit='s'
)
rt_trip_summary['start_time_actual'] = rt_trip_summary['start_time_actual'].dt.tz_localize('US/Pacific')
rt_trip_summary['end_time_actual'] = rt_trip_summary['end_time_actual'].dt.tz_localize('US/Pacific')

In [24]:
rt_trip_summary['time_difference'] = rt_trip_summary['delay_at_end'] - rt_trip_summary['delay_at_start']
rt_trip_summary['duration_rt'] = rt_trip_summary['time_difference'] + rt_trip_summary['duration_sched']
rt_trip_summary['time_difference_pct'] = 100 * rt_trip_summary['time_difference'] / rt_trip_summary['duration_sched']

In [25]:
rt_trip_summary['start_time_actual'].dt.hour.hist()

# Aggregate ChargePoint data to the trip level

In [26]:
# TODO: confusing code here, using both trip_times_df and rt_trip_summary
trip_times_df = rt_trip_summary.set_index(['date', 'vehicle_id', 'trip_id'])
vids = list()
tids = list()
kwhs = list()
mis = list()
cons = list()
dates = list()
n_samples = list()
for (date, vid, tid) in trip_times_df.index:
    # vid_times = trip_times_df.reset_index()
    vid_times = rt_trip_summary[
        (rt_trip_summary['vehicle_id'] == vid) & (rt_trip_summary['date'] == date)
    ].set_index('trip_id')
    vid_vc = vc_df[vc_df['Name'] == vid]

    tid_df = vid_vc[vid_vc['ISO Time'].between(
        vid_times.loc[tid, 'start_time_actual'],
        vid_times.loc[tid, 'end_time_actual'],
        inclusive='both'
    )]
    try:
        # Filter out NAs
        tid_full = tid_df[['ISO Time', 'Energy used (kWh)', 'Distance driven (mi)']].dropna().sort_values('ISO Time')
        kwh_0 = tid_full.iloc[0]['Energy used (kWh)']
        dist_0 = tid_full.iloc[0]['Distance driven (mi)']
        kwh_end = tid_full.iloc[-1]['Energy used (kWh)']
        dist_end = tid_full.iloc[-1]['Distance driven (mi)']
        kwh_used = kwh_end - kwh_0
        mi_driven = dist_end - dist_0
        dates.append(date)
        vids.append(vid)
        tids.append(tid)
        kwhs.append(kwh_used)
        mis.append(mi_driven)
        cons.append(kwh_used / mi_driven)
        n_samples.append(len(tid_df))
    except IndexError:
        # We get here if every row has an NA value for either distance or kWh
        pass

In [27]:
vc_by_trip = pd.DataFrame(
    data={
        'date': dates,
        'vehicle_id': vids,
        'trip_id': tids,
        'kwh': kwhs,
        'miles': mis,
        'kwh_per_mi': cons,
        'n_samples': n_samples
    }
)

## Create an example plot of ChargePoint data

In [21]:
date = datetime.datetime(2024, 3, 2)
vid = '4800'
tid = '635411515'
vid_times = rt_trip_summary[
    (rt_trip_summary['vehicle_id'] == vid) & (rt_trip_summary['date'] == date)
].set_index('trip_id')

vid_vc = vc_df[vc_df['Name'] == vid]
tid_df = vid_vc[vid_vc['ISO Time'].between(
    vid_times.loc[tid, 'start_time_actual'],
    vid_times.loc[tid, 'end_time_actual'],
    inclusive='both'
)].copy()
kwh_0 = tid_df.dropna().iloc[0]['Energy used (kWh)']
dist_0 = tid_df.dropna().iloc[0]['Distance driven (mi)']
tid_df['Energy used (kWh)'] = tid_df['Energy used (kWh)'] - kwh_0
tid_df['Distance driven (mi)'] = tid_df['Distance driven (mi)'] - dist_0

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
secondary = [False, True]
names = ['Energy', 'Distance']
for ix, plot_col in enumerate(['Energy used (kWh)', 'Distance driven (mi)']):
    fig.add_trace(go.Scatter(x=tid_df['ISO Time'], y=tid_df[plot_col], name=names[ix], mode='markers'), secondary_y=secondary[ix])

# Add figure title
fig.update_layout(
    title_text='Bus 4800, Trip 635411515 on 3/2/2024'
)

# Set x-axis title
fig.update_xaxes(title_text='Time')

# Set y-axes titles
fig.update_yaxes(title_text='Energy used (kWh)', secondary_y=False)
fig.update_yaxes(title_text='Distance driven (mi)', secondary_y=True)

fig.write_image('images/chargepoint_trip_example.pdf')
fig.show()

In [22]:
date = datetime.datetime(2024, 3, 15)
vid = '4714'
tid = '635436455'
vid_times = rt_trip_summary[
    (rt_trip_summary['vehicle_id'] == vid) & (rt_trip_summary['date'] == date)
].set_index('trip_id')

vid_vc = vc_df[vc_df['Name'] == vid]
tid_df = vid_vc[vid_vc['ISO Time'].between(
    vid_times.loc[tid, 'start_time_actual'],
    vid_times.loc[tid, 'end_time_actual'],
    inclusive='both'
)].copy()
kwh_0 = tid_df.dropna().iloc[0]['Energy used (kWh)']
dist_0 = tid_df.dropna().iloc[0]['Distance driven (mi)']
tid_df['Energy used (kWh)'] = tid_df['Energy used (kWh)'] - kwh_0
tid_df['Distance driven (mi)'] = tid_df['Distance driven (mi)'] - dist_0

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
secondary = [False, True]
names = ['Energy', 'Distance']
for ix, plot_col in enumerate(['Energy used (kWh)', 'Distance driven (mi)']):
    fig.add_trace(go.Scatter(x=tid_df['ISO Time'], y=tid_df[plot_col], name=names[ix], mode='markers'), secondary_y=secondary[ix])

# Add figure title
fig.update_layout(
    title_text=f'Bus {vid}, Trip {tid} on 3/15/24'
)

# Set x-axis title
fig.update_xaxes(title_text='Time')

# Set y-axes titles
fig.update_yaxes(title_text='Energy used (kWh)', secondary_y=False)
fig.update_yaxes(title_text='Distance driven (mi)', secondary_y=True)

fig.write_image('images/chargepoint_trip_example.pdf')
fig.show()

In [23]:
tid_df['Energy used (kWh)'].iloc[-1], tid_df['Distance driven (mi)'].iloc[-1], tid_df['Energy used (kWh)'].iloc[-1] / tid_df['Distance driven (mi)'].iloc[-1]

# Combine the energy and realtime data

In [29]:
cleaned_trip_data = rt_trip_summary[
    ['date', 'vehicle_id', 'trip_id', 'duration_sched', 'duration_rt', 'time_difference_pct']
].merge(
    vc_by_trip[
        ['date', 'vehicle_id', 'trip_id', 'kwh', 'miles', 'kwh_per_mi']
    ],
    on=['date', 'vehicle_id', 'trip_id']
)

In [30]:
cleaned_trip_data[cleaned_trip_data['time_difference_pct'] >= 0.4*cleaned_trip_data['time_difference_pct'].max()]

In [31]:
# Throw out the above trips with questionable data
cleaned_trip_data = cleaned_trip_data[
    cleaned_trip_data['time_difference_pct'] < 0.4*cleaned_trip_data['time_difference_pct'].max()]
cleaned_trip_data['vehicle_type'] = cleaned_trip_data['vehicle_id'].str[:2].map({'47': '40-foot', '48': '60-foot'})

# Clean up and save the data for further analysis

In [37]:
# Add route info and timing info
data_out = cleaned_trip_data.merge(
    trips_df[['trip_id', 'route_id', 'direction_id']], on='trip_id').merge(
    routes_df[['route_id', 'route_short_name']]).drop(
    columns=['route_id']).merge(
        rt_trip_summary[['date', 'vehicle_id', 'trip_id', 'start_time_actual', 'end_time_actual']], on=['date', 'vehicle_id', 'trip_id']
).rename(
        columns={
            'time_difference_pct': 'duration_difference_pct',
            'route_short_name': 'route',
            'start_time_actual': 'start_time',
            'end_time_actual': 'end_time'
        }
)
# Reorder columns
data_out = data_out[
    ['date', 'vehicle_id', 'vehicle_type', 'trip_id', 'route', 'direction_id', 'start_time', 'end_time',
     'duration_sched', 'duration_rt', 'duration_difference_pct', 'kwh', 'miles', 'kwh_per_mi']
]
data_out.to_csv('../beb_chargers/data/processed/cleaned_trip_data.csv', index=False)

In [39]:
data_out['start_time'].dt.hour.hist()